# AI Business Brochure Generator

- This project use Multi-shot prompting where you provide multiple examples of input-output pairs to help an AI model better understand the pattern or task you want it to perform. Instead of just giving one example (single-shot) or no examples (zero-shot).
- This function allows user to select different language and tones

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [72]:
website_provided = Website("https://www.brooklynkids.org/")

In [73]:
website_provided.links

['#content',
 '/calendar',
 'https://www.brooklynkids.org',
 'https://www.brooklynkids.org/visit/',
 'https://www.brooklynkids.org/visit/',
 'https://www.brooklynkids.org/faq/',
 '/exhibit-type/permanent/',
 '/programs/',
 'https://www.brooklynkids.org/colorlab/',
 'https://www.brooklynkids.org/natures-engineers/',
 '/events/',
 'https://www.brooklynkids.org/movies/',
 'https://www.brooklynkids.org/movies/',
 'https://www.brooklynkids.org/saturday-morning-cartoons/',
 'https://www.brooklynkids.org/birthdays/',
 'https://www.brooklynkids.org/birthdays/',
 'https://www.brooklynkids.org/space-rentals/',
 'https://www.brooklynkids.org/education/',
 'https://www.brooklynkids.org/education/',
 'https://www.brooklynkids.org/school-visits/',
 'https://www.brooklynkids.org/camp-visits/',
 'https://www.brooklynkids.org/summer-camp/',
 'https://www.brooklynkids.org/museum-on-the-go/',
 'https://www.brooklynkids.org/after-school-program/',
 'https://www.brooklynkids.org/teens/',
 'https://www.broo

# Link selection section
### As you can see there are many link in a given website, let's create a block of fucntion to ask GPT which link is relevant


In [74]:
# create a prompt to allow GPT to decide which link from the website is relevant to create a company brochure
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages or Pricing of product/service.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

# create brochure function

In [75]:
# create a function to get details from the links selected by get_links
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result
    
# and then those details will be compose into user prompt to create brochure
def get_brochure_user_prompt(company_name, url, language="English"):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; \
    use this information to build a short {language} brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

# put the two function above together
def create_brochure(company_name, url, language="English", tone ="humorous, entertaining, jokey"):
    system_prompt = f"""You are an assistant that analyzes the contents of several relevant pages from a company website \
    and creates a {tone} about the company for prospective customers, investors and recruits. Respond in markdown.\
    Include details of company culture, customers, product pricing and careers/jobs if you have the information.

    """

    stream= openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language=language)}
          ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [76]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="English", tone="humorous, entertaining, jokey")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'events page', 'url': 'https://www.brooklynkids.org/events/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}]}


# Welcome to the Brooklyn Children's Museum!

**Opening Times:**  
We’re open today from 10 am–5 pm! (because kids need to learn - and where else can you find adult supervision for your sanity?)

---

## Who Are We?  
At the Brooklyn Children's Museum, we’re not just about playdates; we’re about creating lifelong learners! With a mission to promote culture, education, and art through interactive experiences tailored for the curious minds of Brooklyn's children (and their equally curious parents), we've got your weekend plans covered!

### What Do We Offer?
- **Exhibits:** Got a little one who loves dinosaurs? Or perhaps they’re into building things? We’ve got you covered with exhibits like Totally Tots and the Nature's Engineers Makerspace.
  
- **Programs:** STEM education? You bet! Art studios? Absolutely! Join our **ColorLab Art Studio** or the heart-pounding fun in our STEM Makerspace!  

- **Seasonal Events:** Don’t miss our exciting **Black Future Festival**! The fun never stops year-round!

- **Birthday Parties:** Your kid's birthday deserves to be OFF THE CHARTS! Book one of our party packages for a day they (and you) won’t forget! Cake, fun, and no cleanup for you? Yes, please!

---

## Pricing 
- **Tickets:** Bang for your buck - get ready for unlimited exploration for around the cost of a fancy coffee (okay, maybe two)! 
- **Memberships:** Want to become a frequent flyer at BCM? Get **14% Off All Memberships** during select offers! Why pay for one visit when you can plan at least ten?

---

## Join Our Community  
Become a vital part of our BCM family! 

- **Careers:** Love teaching and engaging with kids? Why not work where you can inspire little minds and avoid the coffee machine drama? We’re hiring! Trust us, packing snack bags for summer camps is more fun than it sounds.  
- **Volunteer:** We also welcome enthusiastic helpers ready to put on a smile, unleash some creativity, and keep the giggles rolling.

---

## Your Invitation to Fun!  
Whether you want to soak up some culture, enjoy a day of meticulously supervised chaos, or just let the kids learn about the world in their own way, the Brooklyn Children's Museum is your ultimate destination.

Come explore today, tomorrow, or every day of the week (except Monday & Tuesday - we need a little breathing room)!

**Visit us at:**  
145 Brooklyn Avenue, Brooklyn, NY 11213, USA  

**Follow us on social media:**  
Because nothing says “I’m a responsible adult” like sharing your kids’ adventures online!

---

**See you soon at the BCM — let the imaginative chaos begin!** 🎉✨

In [77]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="English", tone="formal")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}, {'type': 'events page', 'url': 'https://www.brooklynkids.org/events/'}, {'type': 'contact page', 'url': 'https://www.brooklynkids.org/contact/'}]}


# Brooklyn Children's Museum: Discover, Learn, and Play

## About Us
The **Brooklyn Children's Museum (BCM)**, established as one of the first children's museums in the United States, is dedicated to engaging children and families in experiential learning through play, creativity, and exploration. Our mission centers around providing a space where children can discover and learn about the world around them.

## Our Offerings
At BCM, we promote interactive education through a variety of exhibits and programs:
- **Exhibits**: Our engaging exhibits, including "Totally Tots," "AirMaze," and "Neighborhood Nature," inspire curiosity and hands-on interaction.
- **Art & STEM Programs**: Join us at the **ColorLab Art Studio** or the **STEM Makerspace** for innovative art projects and scientific exploration.
- **Movie Screenings**: Enjoy family-friendly movies and cartoons on designated days.
- **Parties & Rentals**: Celebrate birthdays and special events in a fun and engaging environment.

## Education Programs
BCM offers various educational programs for schools, private classes, and summer camps. Our programs include:
- **School Visits**: Tailored school trips that merge fun and education.
- **After School and Teen Programs**: Activities designed to inspire creativity and collaboration.
- **BCM @ Home**: Activity kits and resources to extend learning beyond the museum.

## Culture and Community
The Brooklyn Children's Museum prides itself on fostering an inclusive and welcoming environment. We aim to:
- Encourage creativity and self-expression among children,
- Embrace diversity and support cultural dialogue,
- Engage families in educational and fun activities together.

## Membership and Pricing
Become a BCM member for unlimited free admission and year-round access to special events and programs. Memberships are available at various levels to accommodate all families.

- **General Admission**: Check our website for current pricing and potential discounts.
- **Membership Discounts**: Take advantage of periodic promotions, such as 14% off all memberships, as seen during our events.

## Careers at BCM
At Brooklyn Children’s Museum, we’re always looking for passionate individuals to join our team. Positions vary from educational programming to operational roles. For employment opportunities, visit our **[Career page](#)** for current listings and application details.

## Visit Us
We invite you to bring your family and friends to the Brooklyn Children's Museum and experience the joy of learning through play. We are located at:

**145 Brooklyn Avenue, Brooklyn, NY 11213, USA**  
**Hours of Operation**:  
Wednesday to Sunday: 10 AM - 5 PM  
Closed on Monday and Tuesday  

## Stay Connected
Join the BCM community by subscribing to our newsletter for the latest updates and programs. We are excited to welcome you to a world of discovery and fun!

---

For more information, please visit our website at [Brooklyn Children's Museum](#) or contact us via the **Contact Us** page.

In [78]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="Chinese", tone="formal")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}]}



# 布鲁克林儿童博物馆简介

**欢迎来到布鲁克林儿童博物馆！** 我们是一个专为儿童设计的互动博物馆，提供丰富的教育和娱乐活动，适合各个年龄段的家庭。我们的使命是通过创新的展览和丰富多彩的项目，激发儿童的好奇心和创造力。

## 公司文化
布鲁克林儿童博物馆致力于创造一个包容、乐趣和学习相结合的环境。我们培养社区意识，鼓励家庭积极参与实验、艺术和科学活动。我们的团队由热情洋溢的教育工作者、艺术家和专业人士组成，他们共同努力，为儿童提供最佳的学习体验。

## 客户群体
我们的主要客户是家庭和学校。我们欢迎大大小小的孩子来探索博物馆的各种展览、参加教育项目和享受特殊活动。此外，我们也为学校团体提供定制的学习访问，以增强课堂教学的效果。

## 产品与定价
布鲁克林儿童博物馆提供多种票价选项和会员计划：

- **日票**：一般入场票可在线或现场购买。
- **会员**：成为会员可享受免费进入博物馆的一年期，体验各种家庭活动和展览。
- **特别活动和生日派对**：提供多种空间租赁和定制生日派对服务。

我们现在进行14% 的会员折扣促销活动，欢迎加入我们的社区！

## 职业与就业机会
我们欢迎热爱教育和儿童发展的专业人士加入我们。布鲁克林儿童博物馆提供多样的职业发展机会，包括教育工作者、程序协调员和志愿者职位。我们希望吸引那些充满热情、愿意在动态环境中工作的应聘者。

## 访问信息
- **地址**：145 Brooklyn Avenue, Brooklyn, NY 11213, USA
- **开放时间**：周三至周日，10:00 am - 5:00 pm；周一和周二闭馆。

**加入我们，探索知识的乐趣！** 了解更多信息请访问我们的网站或联系我们。

**联系我们**: [布鲁克林儿童博物馆官方网站](https://www.brooklynkids.org)


